In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

import seaborn as sns
import scipy.stats.stats as st

In [2]:
dataFile = 'restaurant_rating.csv'

In [3]:
dataOG = pd.read_csv(dataFile)

In [4]:
data = dataOG.copy()

In [5]:
data.head(2)

,user_id,business_id,date,review_id,stars,usefulvotes_review,user_name,categories,biz_name,latitude,longitude,business_avg,business_review_count,user_avg,user_review_count
0,rLtl8ZkDX5vH5nAx9C3q5Q,9yKzy9PApeiPPOUJEtnvkg,2011-01-26 00:00:00,fWKvX83p0-ka4JS3dc6E5A,5,5,Jason,"[Breakfast & Brunch, Restaurants]",Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.796954,197
1,SBbftLzfYYKItOMFwOTIJg,9yKzy9PApeiPPOUJEtnvkg,2008-05-04 00:00:00,DASdFe-g0BgfN9J2tanStg,5,1,Jennifer,"[Breakfast & Brunch, Restaurants]",Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.473684,57


In [6]:
data.dtypes

user_id                   object
business_id               object
date                      object
review_id                 object
stars                      int64
usefulvotes_review         int64
user_name                 object
categories                object
biz_name                  object
latitude                 float64
longitude                float64
business_avg             float64
business_review_count      int64
user_avg                 float64
user_review_count          int64
dtype: object

In [7]:
type(data['user_id'][1])

str

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [9]:
spark = SparkSession.builder.appName('SparkRecomm').getOrCreate()

In [10]:
restaurant_rating_spark = dataOG[['user_id', 'business_id', 'review_id', 'stars',
       'usefulvotes_review', 'user_name', 'biz_name', 'latitude',
       'longitude', 'business_avg', 'business_review_count', 'user_avg',
       'user_review_count']]

restaurant_rating_spark.head(2)

,user_id,business_id,review_id,stars,usefulvotes_review,user_name,biz_name,latitude,longitude,business_avg,business_review_count,user_avg,user_review_count
0,rLtl8ZkDX5vH5nAx9C3q5Q,9yKzy9PApeiPPOUJEtnvkg,fWKvX83p0-ka4JS3dc6E5A,5,5,Jason,Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.796954,197
1,SBbftLzfYYKItOMFwOTIJg,9yKzy9PApeiPPOUJEtnvkg,DASdFe-g0BgfN9J2tanStg,5,1,Jennifer,Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.473684,57


In [11]:
restaurant_rating_spark.head(2)

,user_id,business_id,review_id,stars,usefulvotes_review,user_name,biz_name,latitude,longitude,business_avg,business_review_count,user_avg,user_review_count
0,rLtl8ZkDX5vH5nAx9C3q5Q,9yKzy9PApeiPPOUJEtnvkg,fWKvX83p0-ka4JS3dc6E5A,5,5,Jason,Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.796954,197
1,SBbftLzfYYKItOMFwOTIJg,9yKzy9PApeiPPOUJEtnvkg,DASdFe-g0BgfN9J2tanStg,5,1,Jennifer,Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.473684,57


In [82]:
bizID_dict = {}
numBiz = len(restaurant_rating_spark['business_id'].unique())
#restaurant_rating_spark['business_id_int'] = np.zeros((restaurant_rating_spark.shape[0], 1)).astype(np.int)
bizID_BOW = []
bizEncoder = 0
for n in np.arange(restaurant_rating_spark.shape[0]):
    bizID = restaurant_rating_spark.loc[n, 'business_id']
    if bizID not in bizID_BOW:
        bizID_BOW.append(bizID)
        bizID_dict[bizID] = bizEncoder
        #restaurant_rating_spark.loc[n, 'business_id_int'] = bizEncoder
        bizEncoder += 1
    #else:
    #    restaurant_rating_spark.loc[n, 'business_id_int'] = bizID_dict[bizID]

In [77]:
restaurant_rating_spark['business_id_int'] = restaurant_rating_spark['business_id'].map(bizID_dict)
#restaurant_rating_spark['business_id_int'] = np.zeros((restaurant_rating_spark.shape[0], 1)).astype(np.int)

In [84]:
restaurant_rating_spark.head(2)

,user_id,business_id,review_id,stars,usefulvotes_review,user_name,biz_name,latitude,longitude,business_avg,business_review_count,user_avg,user_review_count,business_id_int,user_id_int
0,rLtl8ZkDX5vH5nAx9C3q5Q,9yKzy9PApeiPPOUJEtnvkg,fWKvX83p0-ka4JS3dc6E5A,5,5,Jason,Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.796954,197,0,0
1,SBbftLzfYYKItOMFwOTIJg,9yKzy9PApeiPPOUJEtnvkg,DASdFe-g0BgfN9J2tanStg,5,1,Jennifer,Morning Glory Cafe,33.390792,-112.012504,3.87156,109,3.473684,57,0,1


In [15]:
usrID_dict = {}
numUsr = len(restaurant_rating_spark['user_id'].unique())

usrID_BOW = []
usrEncoder = 0
for n in np.arange(restaurant_rating_spark.shape[0]):
    usrID = restaurant_rating_spark.loc[n, 'user_id']
    if usrID not in usrID_BOW:
        usrID_BOW.append(usrID)
        usrID_dict[usrID] = usrEncoder
        usrEncoder += 1

In [16]:
restaurant_rating_spark['user_id_int'] = restaurant_rating_spark['user_id'].map(usrID_dict)

In [17]:
len(restaurant_rating_spark['user_id_int'].unique())

34789

In [18]:
len(restaurant_rating_spark['business_id_int'].unique())

4503

In [83]:
restaurant_rating_spark.to_csv('restaurant_rating_spark.csv')

In [20]:
with open('restaurant_rating_spark.csv') as f:
    next(f)
    for i, lines in enumerate(f):
        if i < 2:
            line = lines.split(',')
            print(line)
            print(len(line))
        else:
            break

['0', 'rLtl8ZkDX5vH5nAx9C3q5Q', '9yKzy9PApeiPPOUJEtnvkg', 'fWKvX83p0-ka4JS3dc6E5A', '5', '5', 'Jason', 'Morning Glory Cafe', '33.390792', '-112.012504', '3.8715596330275233', '109', '3.796954314720812', '197', '0', '0\n']
16
['1', 'SBbftLzfYYKItOMFwOTIJg', '9yKzy9PApeiPPOUJEtnvkg', 'DASdFe-g0BgfN9J2tanStg', '5', '1', 'Jennifer', 'Morning Glory Cafe', '33.390792', '-112.012504', '3.8715596330275233', '109', '3.4736842105263164', '57', '0', '1\n']
16


In [85]:
def mapper(lines):
    lines = lines.replace("'","").replace("\n","")
    line = lines.split(',')
    element = Row(
            caseID = int(line[0]),
            userID = str(line[1]),
            bizID = str(line[2]), 
            reviewID = str(line[3]),
            stars = int(line[4]),
            userfulvotes_review = int(line[5]), 
            userName = str(line[6]), 
            bizName = str(line[7]), 
            #latitude = float(line[8]), 
            #longitude = float(line[9]), 
            biz_avg = float(line[10]), 
            bizReviewCnt = float(line[11]), 
            user_avg = float(line[12]),
            userReviewCnt = float(line[13]),
            bizID_int = int(line[14]),
            userID_int = int(line[15].split('\n')[0]))
    return element

In [86]:
lines = spark.sparkContext.textFile('restaurant_rating_spark.csv')

In [87]:
lines = lines.filter(lambda l: not '_id' in l)
lines.take(2)

['0,rLtl8ZkDX5vH5nAx9C3q5Q,9yKzy9PApeiPPOUJEtnvkg,fWKvX83p0-ka4JS3dc6E5A,5,5,Jason,Morning Glory Cafe,33.390792,-112.012504,3.8715596330275233,109,3.796954314720812,197,0,0',
 '1,SBbftLzfYYKItOMFwOTIJg,9yKzy9PApeiPPOUJEtnvkg,DASdFe-g0BgfN9J2tanStg,5,1,Jennifer,Morning Glory Cafe,33.390792,-112.012504,3.8715596330275233,109,3.4736842105263164,57,0,1']

In [88]:
dataSpark = lines.map(mapper)
dataSpark.take(1)

[Row(bizID='9yKzy9PApeiPPOUJEtnvkg', bizID_int=0, bizName='Morning Glory Cafe', bizReviewCnt=109.0, biz_avg=3.8715596330275233, caseID=0, reviewID='fWKvX83p0-ka4JS3dc6E5A', stars=5, userID='rLtl8ZkDX5vH5nAx9C3q5Q', userID_int=0, userName='Jason', userReviewCnt=197.0, user_avg=3.796954314720812, userfulvotes_review=5)]

In [89]:
test = dataSpark.take(1)

In [90]:
test[0]

Row(bizID='9yKzy9PApeiPPOUJEtnvkg', bizID_int=0, bizName='Morning Glory Cafe', bizReviewCnt=109.0, biz_avg=3.8715596330275233, caseID=0, reviewID='fWKvX83p0-ka4JS3dc6E5A', stars=5, userID='rLtl8ZkDX5vH5nAx9C3q5Q', userID_int=0, userName='Jason', userReviewCnt=197.0, user_avg=3.796954314720812, userfulvotes_review=5)

## create rating Spark dataframe - use ALS from ml not mllib

In [91]:
rating_spark_DF = spark.createDataFrame(dataSpark)

In [92]:
(training, valid, test) = rating_spark_DF.randomSplit([0.6, 0.2, 0.2])

In [93]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import itertools

In [97]:
als = ALS(maxIter = 15, regParam = 0.01, userCol = 'userID_int', itemCol = 'bizID_int', ratingCol = 'stars')

In [98]:
model = als.fit(training)

In [99]:
training.take(1)

[Row(bizID='-3xbryp44xhpN4BohxXDdQ', bizID_int=12, bizName='The Keg Steakhouse & Bar', bizReviewCnt=61.0, biz_avg=3.6557377049180326, caseID=1484, reviewID='VY-BRys0309NJ6Pk8NcqFA', stars=4, userID='0ouMuScWJ3GuNtWEmmsziw', userID_int=1315, userName='Kate', userReviewCnt=49.0, user_avg=4.224489795918367, userfulvotes_review=4)]

In [107]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='stars', predictionCol='prediction')

In [111]:
predictions_training = model.transform(training).cache()

In [112]:
rmse_train = evaluator.evaluate(predictions_training)

In [113]:
rmse_train

0.23853486095903864

In [105]:
predictions_test = model.transform(test).cache()

In [102]:
rmse_test = evaluator.evaluate(predictions_test)

In [103]:
predictions.take(1)
#rmse

[Row(bizID='0udEgNqy5rLR5pZ4kD19Og', bizID_int=148, bizName='Thai Basil', bizReviewCnt=145.0, biz_avg=4.048275862068966, caseID=18268, reviewID='3HBDLBwbGFCmJnkM1tv8Mg', stars=4, userID='z2aIK__6KMpQBx1czvv8BA', userID_int=1395, userName='Alex', userReviewCnt=17.0, user_avg=3.941176470588236, userfulvotes_review=1, prediction=3.6396515369415283)]

In [104]:
predictions.groupBy('prediction').count().orderBy('count', ascending=False).take(1)

[Row(prediction=nan, count=5049)]

## create spark DataFrame

In [118]:
schemaRating = spark.createDataFrame(dataSpark).cache()
schemaRating.createOrReplaceTempView('rating')

In [119]:
partUserBiz = spark.sql('SELECT * FROM rating where bizReviewCnt > 150 and userReviewCnt > 60')

In [120]:
partUserBiz.take(3)

[Row(bizID='eIxSLxzIlfExI6vgAbn2JA', bizID_int=4, bizName='Lobbys Beef Burgers Dogs', bizReviewCnt=173.0, biz_avg=4.283236994219654, caseID=395, reviewID='z4G1g3VcdfigCUtVDFRL1w', stars=4, userID='rLtl8ZkDX5vH5nAx9C3q5Q', userID_int=0, userName='Jason', userReviewCnt=197.0, user_avg=3.796954314720812, userfulvotes_review=5),
 Row(bizID='eIxSLxzIlfExI6vgAbn2JA', bizID_int=4, bizName='Lobbys Beef Burgers Dogs', bizReviewCnt=173.0, biz_avg=4.283236994219654, caseID=397, reviewID='480M-xDpRaA1gCVCqIG4WA', stars=4, userID='C6IOtaaYdLIT5fWd7ZYIuA', userID_int=375, userName='Daren', userReviewCnt=209.0, user_avg=3.784688995215311, userfulvotes_review=14),
 Row(bizID='eIxSLxzIlfExI6vgAbn2JA', bizID_int=4, bizName='Lobbys Beef Burgers Dogs', bizReviewCnt=173.0, biz_avg=4.283236994219654, caseID=399, reviewID='q_T6FtScTul_yYIGyrgTxA', stars=3, userID='mlBC3pN9GXlUUfQi1qBBZA', userID_int=2, userName='Jennifer', userReviewCnt=84.0, user_avg=3.702380952380953, userfulvotes_review=0)]

In [121]:
#partUserBiz = partUserBiz.collect()

## top restaurants with being rated certain times

In [122]:
userbizRating_subset = spark.sql('SELECT caseID, userID_int, bizID_int, stars FROM rating')

In [123]:
userbizRating_subset.take(3)

[Row(caseID=0, userID_int=0, bizID_int=0, stars=5),
 Row(caseID=1, userID_int=1, bizID_int=0, stars=5),
 Row(caseID=2, userID_int=2, bizID_int=0, stars=2)]

In [124]:
bizIDbizName_subset = spark.sql('SELECT DISTINCT bizID, bizName FROM rating')
bizID_int_bizName_subset = spark.sql('SELECT DISTINCT bizID_int, bizName FROM rating')

In [125]:
bizIDbizName_subset.take(3)

[Row(bizID='HPixSiN2eEx9cI-z3tQJSQ', bizName='Cafe Roma'),
 Row(bizID='X5QTGpPfqXFtmtizsGAksw', bizName='Mucho Gusto Taqueria Mexican Bistro'),
 Row(bizID='yYbd9P1KmlPSKmQxo68n_g', bizName='Pork on a Fork')]

In [126]:
bizID_int_bizName_subset.take(3)

[Row(bizID_int=236, bizName='Barros Pizza'),
 Row(bizID_int=483, bizName='Sautee'),
 Row(bizID_int=500, bizName='Luna Pizza & Restaurant')]

Top restaurants ranked by their average ratings

In [127]:
import pyspark.sql.functions as func
rankBiz = userbizRating_subset.groupBy(['bizID_int', 'stars']).agg(func.mean('stars'))\
          .orderBy('avg(stars)', ascending=False)

In [128]:
rankBiz.take(1)

[Row(bizID_int=506, stars=5, avg(stars)=5.0)]

we can simply left join the ranked restaurants to table for maping restaurantID to restaurantName

In [129]:
rankBiz.join(bizID_int_bizName_subset, on = 'bizID_int', how = 'left_outer').take(10)

[Row(bizID_int=251, stars=5, avg(stars)=5.0, bizName='Cafe Rio'),
 Row(bizID_int=668, stars=5, avg(stars)=5.0, bizName='Black Bear Diner'),
 Row(bizID_int=1176, stars=5, avg(stars)=5.0, bizName='Tarbells'),
 Row(bizID_int=1208, stars=5, avg(stars)=5.0, bizName='Yogis Grill'),
 Row(bizID_int=1340, stars=5, avg(stars)=5.0, bizName='Los Taquitos'),
 Row(bizID_int=1714, stars=5, avg(stars)=5.0, bizName='Francos Italian Caffé'),
 Row(bizID_int=1880, stars=5, avg(stars)=5.0, bizName='Rockfish Seafood Grill'),
 Row(bizID_int=2095, stars=5, avg(stars)=5.0, bizName='China China Restaurant'),
 Row(bizID_int=2442, stars=5, avg(stars)=5.0, bizName='Schlotzskys'),
 Row(bizID_int=2629, stars=5, avg(stars)=5.0, bizName='Taco Bell')]

## split subset into training, validation and test

In [130]:
numCase = partUserBiz.count()
numTrain = int(0.6 * numCase)
numValid = int(0.2 * numCase)
numTest  = int(0.2 * numCase)
print('split small dataset with {0} cases to {1} training, {2} valdation and {3} test'
      .format(numCase, numTrain, numValid, numTest))

split small dataset with 6165 cases to 3699 training, 1233 valdation and 1233 test


In [131]:
numPartitions = 4

In [132]:
train_set = spark.sql('SELECT userID_int as user, bizID_int as product, stars FROM rating WHERE caseID < 3699')\
            .repartition(numPartitions).cache()

print(train_set.take(1))
print(train_set.count())

[Row(user=1, product=0, stars=5)]
3699


In [133]:
valid_set = spark.sql('SELECT userID_int as user, bizID_int as product, stars FROM rating WHERE 3699 <= caseID AND caseID < 4932')\
            .repartition(numPartitions).cache()
print(valid_set.take(4))
print(valid_set.count())

[Row(user=2876, product=33, stars=5), Row(user=2737, product=33, stars=4), Row(user=2881, product=33, stars=3), Row(user=2885, product=33, stars=4)]
1233


In [134]:
test_set = spark.sql('SELECT userID_int as user, bizID_int as product, stars FROM rating WHERE 4932 <= caseID AND caseID < 6165')\
            .repartition(numPartitions).cache()
print(test_set.take(4))
print(test_set.count())

[Row(user=947, product=40, stars=3), Row(user=2411, product=40, stars=4), Row(user=2413, product=40, stars=4), Row(user=1271, product=40, stars=3)]
1233


## train ALS model - Spark RDD

In [135]:
from pyspark.mllib.recommendation import ALS
import itertools

Three hyper-parameters: 

1. rank -- how many latent features we are using to construct restaurant matrix and user matrix. Latent features can be interpreted as cleaningness, spicy, comfortability, etc, which are somehow being made up by developer.

2. regularized rate -- to prevent model fitted from overfitting, it is necessary to introduce a regularized parameter.

3. iterations -- feature values are recrusively updated by optimizing cost function (square error) through gradient descent, so it need to be specified for running optimization solver.

In [136]:
ranks = [5, 10, 15]
lambdas = [0.1, 0.3, 0.5]
iterations = [3, 6, 9]

In [137]:
bestValidationRmse = np.float('inf')

In [138]:
bestALS = None

In [139]:
bestRank = 0

In [140]:
bestLambda = -1.0

In [141]:
bestIterNum = -1

use RMSE as metric of model fitted

In [142]:
from math import sqrt
from operator import add

In [143]:
def rmseEval(model, data, validCnt):
    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    predictionsAndRatings = predictions.map(lambda x:((x[0], x[1], x[2])))\
                            .join((data.map(lambda x:((x[0], x[1], x[2]))))).values()
    rmse = sqrt(predictionsAndRatings.map(lambda x: (x[0] - x[1]) ** 2).reduce(add) / float(validCnt))
    return rmse

In [167]:
model = ALS.train(ratings=train_set, rank=15, lambda_=0.01, iterations=15)

In [168]:
train_ID_Int_set = spark.sql('SELECT userID_int, bizID_int FROM rating WHERE 3699 > caseID')\
            .repartition(numPartitions).cache()
predictions_training = model.predictAll(train_ID_Int_set.rdd)

In [169]:
#valid_ID_Int_set = spark.sql('SELECT userID_int, bizID_int FROM rating WHERE 3699 <= caseID AND caseID < 4932')\
#            .repartition(numPartitions).cache()
#predictions_training = model.predictAll(valid_ID_Int_set.rdd)

### comparison between semi-self implement ALS and built-in ALS

built in rmse is significant smaller than the semi-self implement, this is due to the the former uses random split to construct the training, validation and test dataset, while in the semi-self implementation the split is performed based the caseID that is recording ascending order and it is highly possible that a dozen of caseIDs are rating for one restaurant. Hence, the diversity of training set is relatively low and the shuffle could be the solution of this problem.

In [170]:
rmseEval(model, train_set.rdd, numTrain)

13.842234699501184

## NaN values in predictions of validation

For those users are literatually non-existing in training-set, when they come out in validation prediction stage it is reasonable that the model will return NAN as the predicted rating since they are never seen from modelling stage. Similarity measure can be adopted in this scenario and recommend restaurants to those users having NAN predicted rating via their similar user.

An userful discussion can be found in [here](https://issues.apache.org/jira/browse/SPARK-14489).

In [1]:
'''
for rank, lambdaVal, numIter in itertools.product(ranks, lambdas, iterations):
    # fit the ALS model
    model = ALS.train(ratings = train_set, rank = rank, lambda_ = lambdaVal, iterations = numIter)
    
    # measure the RMSE on validation set
    validationRmse = rmseEval(model, valid_set, numValid)
'''    

'\nfor rank, lambdaVal, numIter in itertools.product(ranks, lambdas, iterations):\n    # fit the ALS model\n    model = ALS.train(ratings = train_set, rank = rank, lambda_ = lambdaVal, iterations = numIter)\n    \n    # measure the RMSE on validation set\n    validationRmse = rmseEval(model, valid_set, numValid)\n'